# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,153.850498,0.008105,1234,153.858603,153.842392,307.717206,307.684784
6,323.396214,0.334357,1234,323.730571,323.061857,647.461143,646.123714
7,345.559456,-0.165628,1234,345.393828,345.725085,690.787656,691.450170
8,764.694856,0.540729,1234,765.235586,764.154127,1530.471171,1528.308254
9,116.876682,0.116807,1234,116.993489,116.759875,233.986978,233.519750
10,585.019172,0.727909,1234,585.747081,584.291263,1171.494162,1168.582527
11,58.730362,-0.295378,1234,58.434984,59.025741,116.869967,118.051481
12,9.463895,0.518903,1234,9.982798,8.944993,19.965595,17.889985
13,112.276979,-0.578934,1234,111.698044,112.855913,223.396088,225.711826
14,15.457001,0.393990,1234,15.850991,15.063011,31.701983,30.126022


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-153.858603,153.842392
6,-323.730571,323.061857
7,-345.393828,345.725085
8,-765.235586,764.154127
9,-116.993489,116.759875
10,-585.747081,584.291263
11,-58.434984,59.025741
12,-9.982798,8.944993
13,-111.698044,112.855913
14,-15.850991,15.063011


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-153.858603,153.842392
6,-323.730571,323.061857
7,-345.393828,345.725085
8,-765.235586,764.154127
9,-116.993489,116.759875
10,-585.747081,584.291263
11,-58.434984,59.025741
12,-9.982798,8.944993
13,-111.698044,112.855913
14,-15.850991,15.063011


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-153.85860295,  153.84239215],
       [-323.73057146,  323.06185688],
       [-345.39382788,  345.72508486],
       [-765.23558564,  764.15412698],
       [-116.99348881,  116.75987481],
       [-585.74708108,  584.29126348],
       [ -58.43498368,   59.02574067],
       [  -9.98279774,    8.94499253],
       [-111.69804418,  112.85591291],
       [ -15.8509914 ,   15.06301119]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,305.717206,-307.684784
6,645.461143,-646.123714
7,688.787656,-691.450170
8,1528.471171,-1528.308254
9,231.986978,-233.519750
10,1169.494162,-1168.582527
11,114.869967,-118.051481
12,17.965595,-17.889985
13,221.396088,-225.711826
14,29.701983,-30.126022


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.153850,0.504053
6,0.323396,0.667179
7,0.345559,0.417186
8,0.764695,0.770365
9,0.116877,0.558404
10,0.585019,0.863954
11,0.058730,0.352311
12,0.009464,0.759451
13,0.112277,0.210533
14,0.015457,0.696995


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,153.850498,0.008105
6,323.396214,0.334357
7,345.559456,-0.165628
8,764.694856,0.540729
9,116.876682,0.116807
10,585.019172,0.727909
11,58.730362,-0.295378
12,9.463895,0.518903
13,112.276979,-0.578934
14,15.457001,0.393990


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
